<a href="https://colab.research.google.com/github/ruksharahmed7/simple_nn_chatbot/blob/main/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

#Optional: move to the desired location:
#%cd drive/My Drive/DIRECTORY_IN_YOUR_DRIVE

Mounted at /content/drive


In [2]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('drive/My Drive/Omdena/EgyptChatbot/chatbot_tuts/PythonChatbotTut/intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [5]:

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))


In [6]:

pickle.dump(words, open('drive/My Drive/Omdena/EgyptChatbot/chatbot_tuts/PythonChatbotTut/words.pkl', 'wb'))
pickle.dump(classes, open('drive/My Drive/Omdena/EgyptChatbot/chatbot_tuts/PythonChatbotTut/classes.pkl', 'wb'))

In [7]:
training = []
outputEmpty = [0] * len(classes)
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

In [9]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)
model.save('drive/My Drive/Omdena/EgyptChatbot/chatbot_tuts/PythonChatbotTut/chatbot_model.h5', hist)
print('Done')


Epoch 1/200
4/4 [==============================] - 5s 6ms/step - loss: 1.4090 - accuracy: 0.3333
Epoch 2/200
4/4 [==============================] - 0s 5ms/step - loss: 1.3374 - accuracy: 0.5000
Epoch 3/200
4/4 [==============================] - 0s 4ms/step - loss: 1.2966 - accuracy: 0.5556
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 1.2071 - accuracy: 0.6111
Epoch 5/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1912 - accuracy: 0.6667
Epoch 6/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1339 - accuracy: 0.6111
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 1.2090 - accuracy: 0.5556
Epoch 8/200
4/4 [==============================] - 0s 5ms/step - loss: 1.1606 - accuracy: 0.5556
Epoch 9/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1513 - accuracy: 0.5556
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 1.0758 - accuracy: 0.6667
Epoch 11/200
4/4 [===========

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4736      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 260       
                                                                 
Total params: 13,252
Trainable params: 13,252
Non-trainable params: 0
_________________________________________________________________
